In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

LLMChain

In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

llm = OpenAI(model_name="text-davinci-003", temperature=0)


prompt_template = "What is a word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)


# # alternatively:

# template = """
# What is a word to replace the following: {word}
# """

# prompt = PromptTemplate(
#     template=template,
#     input_variables=["word"],   
# )

# llm_chain = LLMChain(
#     llm=llm,
#     prompt=prompt
# )

/home/cvd/yes/envs/tf/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


alternatives ways to run the chain

In [3]:
llm_chain("artificial")

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [4]:
# multiple inputs

input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nAutomaton'}]

In [6]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nAutomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 46, 'completion_tokens': 13, 'prompt_tokens': 33}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('c61c338a-e99c-4d74-9063-f391301d9349')), RunInfo(run_id=UUID('560d7d27-9022-4677-bb7f-6c3fd741ef4d')), RunInfo(run_id=UUID('6d425e6b-1504-4baa-8991-031aa3054927'))])

The next method we will discuss is .predict(). (which could be used interchangeably with .run()) Its best use case is to pass multiple inputs for a single prompt. However, it is possible to use it with one input variable as well. The following prompt will pass both the word we want a substitute for and the context the model must consider.

In [5]:
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")
# or llm_chain.run(word="fan", context="object")

'\n\nventilator'

In [6]:
llm_chain.predict(word="fan", context="humans")
# or llm_chain.run(word="fan", context="humans")

'\n\nAdmirer'

##### Parsers

In [7]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['Synthetic',
 'Manufactured',
 'Imitation',
 'Fabricated',
 'Fake',
 'Simulated',
 'Artificial Intelligence',
 'Automated',
 'Constructed',
 'Programmed',
 'Mechanical',
 'Processed',
 'Algorithmic',
 'Generated.']

#### Conversational Chain (Memory)

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="Which is the capital of Bulgaria? Give me just tha name.")

' The capital of Bulgaria is Sofia.'

In [11]:
conversation.predict(input="Now many people live there?")

' According to the latest census, the population of Sofia is 1,241,675 people.'

##### Sequential Chain

In [13]:
# from langchain.chains import SimpleSequentialChain

# overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two])

##### Custom Chain

The LangChain library has several predefined chains for different applications like Transformation Chain, LLMCheckerChain, LLMSummarizationCheckerChain, and OpenAPI Chain, which all share the same characteristics mentioned in previous sections. It is also possible to define your chain for any custom task. In this section, we will create a chain that returns a word's meaning and then suggests a replacement.

It starts by defining a class that inherits most of its functionalities from the Chain class. Then, the following three methods must be declared depending on the use case. The input_keys and output_keys methods let the model know what it should expect, and a _call method runs each chain and merges their outputs.

In [14]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

Then, we will declare each chain individually using the LLMChain class. Lastly, we call our custom chain ConcatenateChain to merge the results of the chain_1 and chain_2.

In [15]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is not natural or made by humans, but rather created or produced by artificial means.

Synthetic
